<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>（大規模言語モデルをスクラッチから構築）書籍の補足コード<br>
著者：<a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>コードリポジトリ：<a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# Tiktoken BPEトークナイザーへの新しいトークンの追加

- このノートブックでは、既存のBPEトークナイザーを拡張する方法を説明します。特に、人気の高い[tiktoken](https://github.com/openai/tiktoken)実装での方法に焦点を当てます
- トークン化の一般的な紹介については、[第2章](https://github.com/rasbt/LLMs-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb)およびBPE from Scratch [link]チュートリアルを参照してください
- 例えば、GPT-2トークナイザーがあり、次のテキストをエンコードしたいとします：

In [1]:
import tiktoken

base_tokenizer = tiktoken.get_encoding("gpt2")
sample_text = "Hello, MyNewToken_1 is a new token. <|endoftext|>"

token_ids = base_tokenizer.encode(sample_text, allowed_special={"<|endoftext|>"})
print(token_ids)

[15496, 11, 2011, 3791, 30642, 62, 16, 318, 257, 649, 11241, 13, 220, 50256]


- 各トークンIDを繰り返し処理することで、語彙を介してトークンIDがどのようにデコードされるかをよりよく理解できます：

In [2]:
for token_id in token_ids:
    print(f"{token_id} -> {base_tokenizer.decode([token_id])}")

15496 -> Hello
11 -> ,
2011 ->  My
3791 -> New
30642 -> Token
62 -> _
16 -> 1
318 ->  is
257 ->  a
649 ->  new
11241 ->  token
13 -> .
220 ->  
50256 -> <|endoftext|>


- 上記のように、`"MyNewToken_1"`は5つの個別のサブワードトークンに分解されています -- これは未知の単語を扱うBPEの通常の動作です
- しかし、他の単語や`"<|endoftext|>"`と同様に、単一のトークンとしてエンコードしたい特殊トークンである場合を想定してください。このノートブックではその方法を説明します

&nbsp;
## 1. 特殊トークンの追加

- 新しいトークンは特殊トークンとして追加する必要があることに注意してください。理由は、トークナイザーの訓練プロセス中に作成される新しいトークンの「マージ」がないためです -- たとえそれらを持っていたとしても、既存のトークン化スキームを壊さずにそれらを組み込むことは非常に困難です（「マージ」を理解するには、BPE from scratchノートブック [link]を参照してください）
- 2つの新しいトークンを追加したいとします：

In [ ]:
# カスタムトークンとそのトークンIDを定義
custom_tokens = ["MyNewToken_1", "MyNewToken_2"]
custom_token_ids = {
    token: base_tokenizer.n_vocab + i for i, token in enumerate(custom_tokens)
}

- 次に、特殊トークンを保持するカスタム`Encoding`オブジェクトを次のように作成します：

In [ ]:
# 拡張トークンを含む新しいEncodingオブジェクトを作成
extended_tokenizer = tiktoken.Encoding(
    name="gpt2_custom",
    pat_str=base_tokenizer._pat_str,
    mergeable_ranks=base_tokenizer._mergeable_ranks,
    special_tokens={**base_tokenizer._special_tokens, **custom_token_ids},
)

- 以上で、サンプルテキストをエンコードできることを確認できます：

- ご覧のとおり、新しいトークン`50257`と`50258`が出力にエンコードされています：

In [5]:
special_tokens_set = set(custom_tokens) | {"<|endoftext|>"}

token_ids = extended_tokenizer.encode(
    "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>",
    allowed_special=special_tokens_set
)
print(token_ids)

[36674, 2420, 351, 220, 50257, 290, 220, 50258, 13, 220, 50256]


- 繰り返しになりますが、トークンごとに見ることもできます：

In [6]:
for token_id in token_ids:
    print(f"{token_id} -> {extended_tokenizer.decode([token_id])}")

36674 -> Sample
2420 ->  text
351 ->  with
220 ->  
50257 -> MyNewToken_1
290 ->  and
220 ->  
50258 -> MyNewToken_2
13 -> .
220 ->  
50256 -> <|endoftext|>


- 上記のとおり、トークナイザーの更新に成功しました
- ただし、事前訓練済みLLMで使用するには、LLMの埋め込み層と出力層も更新する必要があります。これについては次のセクションで説明します

&nbsp;
## 2. 事前訓練済みLLMの更新

- このセクションでは、トークナイザーを更新した後、既存の事前訓練済みLLMを更新する方法を見ていきます
- このために、メインブックで使用されているオリジナルの事前訓練済みGPT-2モデルを使用します

&nbsp;
### 2.1 事前訓練済みGPTモデルのロード

In [ ]:
from llms_from_scratch.ch05 import download_and_load_gpt2
# llms_from_scratchのインストール手順については、以下を参照してください：
# https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

In [ ]:
from llms_from_scratch.ch04 import GPTModel
# llms_from_scratchのインストール手順については、以下を参照してください：
# https://github.com/rasbt/LLMs-from-scratch/tree/main/pkg

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # 語彙サイズ
    "context_length": 256, # 短縮されたコンテキスト長（元：1024）
    "emb_dim": 768,        # 埋め込み次元
    "n_heads": 12,         # アテンションヘッド数
    "n_layers": 12,        # レイヤー数
    "drop_rate": 0.1,      # ドロップアウト率
    "qkv_bias": False      # クエリ・キー・バリューバイアス
}

# コンパクトさのためにモデル設定を辞書で定義
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# ベース設定をコピーして特定のモデル設定で更新
model_name = "gpt2-small (124M)"  # モデル名の例
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})

gpt = GPTModel(NEW_CONFIG)
gpt.eval();

### 2.2 事前訓練済みGPTモデルの使用

- 次に、オリジナルと新しいトークナイザーを使ってトークン化する以下のサンプルテキストを検討します：

In [9]:
sample_text = "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>"

original_token_ids = base_tokenizer.encode(
    sample_text, allowed_special={"<|endoftext|>"}
)

In [10]:
new_token_ids = extended_tokenizer.encode(
    "Sample text with MyNewToken_1 and MyNewToken_2. <|endoftext|>",
    allowed_special=special_tokens_set
)

- では、オリジナルのトークンIDをGPTモデルに与えてみましょう：

In [11]:
import torch

with torch.no_grad():
    out = gpt(torch.tensor([original_token_ids]))

print(out)

tensor([[[ 0.2204,  0.8901,  1.0138,  ...,  0.2585, -0.9192, -0.2298],
         [ 0.6745, -0.0726,  0.8218,  ..., -0.1768, -0.4217,  0.0703],
         [-0.2009,  0.0814,  0.2417,  ...,  0.3166,  0.3629,  1.3400],
         ...,
         [ 0.1137, -0.1258,  2.0193,  ..., -0.0314, -0.4288, -0.1487],
         [-1.1983, -0.2050, -0.1337,  ..., -0.0849, -0.4863, -0.1076],
         [-1.0675, -0.5905,  0.2873,  ..., -0.0979, -0.8713,  0.8415]]])


- 上記のように、これは問題なく動作します（注：コードはシンプルにするため、出力をテキストに変換せずに生の出力を示しています。詳細については、第5章 [link] セクション5.3.3の`generate`関数を確認してください）

- 更新されたトークナイザーで生成されたトークンIDで同じことを試すとどうなるでしょうか？

```python
with torch.no_grad():
    gpt(torch.tensor([new_token_ids]))

print(out)

...
# IndexError: インデックスが範囲外です
```

- ご覧のとおり、これはインデックスエラーになります
- その理由は、GPTモデルが入力埋め込み層と出力層を介して固定語彙サイズを期待しているためです：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/extend-tiktoken/gpt-updates.webp" width="400px">

&nbsp;
### 2.3 埋め込み層の更新

- 埋め込み層の更新から始めましょう
- まず、埋め込み層には語彙サイズに対応する50,257個のエントリがあることに注意してください：

In [12]:
gpt.tok_emb

Embedding(50257, 768)

- この埋め込み層に2つのエントリを追加して拡張したいと思います
- 簡単に言えば、より大きなサイズの新しい埋め込み層を作成し、古い埋め込み層の値をコピーします

In [ ]:
num_tokens, emb_size = gpt.tok_emb.weight.shape
new_num_tokens = num_tokens + 2

# 新しい埋め込み層を作成
new_embedding = torch.nn.Embedding(new_num_tokens, emb_size)

# 古い埋め込み層から重みをコピー
new_embedding.weight.data[:num_tokens] = gpt.tok_emb.weight.data

# モデル内の古い埋め込み層を新しいものに置き換え
gpt.tok_emb = new_embedding

print(gpt.tok_emb)

- 上記のとおり、埋め込み層が増加しました

&nbsp;
### 2.4 出力層の更新

- 次に、埋め込み層と同様に語彙サイズに対応する50,257の出力特徴を持つ出力層を拡張する必要があります（ちなみに、PyTorchのLinearとEmbedding層の類似性について議論しているボーナス資料も役立つかもしれません）

In [14]:
gpt.out_head

Linear(in_features=768, out_features=50257, bias=False)

- 出力層の拡張手順は、埋め込み層の拡張と同様です：

In [ ]:
original_out_features, original_in_features = gpt.out_head.weight.shape

# 新しい出力特徴数を定義（例：2つの新しいトークンを追加）
new_out_features = original_out_features + 2

# 拡張された出力サイズで新しい線形層を作成
new_linear = torch.nn.Linear(original_in_features, new_out_features)

# 元の線形層から重みとバイアスをコピー
with torch.no_grad():
    new_linear.weight[:original_out_features] = gpt.out_head.weight
    if gpt.out_head.bias is not None:
        new_linear.bias[:original_out_features] = gpt.out_head.bias

# 元の線形層を新しいものに置き換え
gpt.out_head = new_linear

print(gpt.out_head)

- まず、この更新されたモデルをオリジナルのトークンIDで試してみましょう：

In [16]:
with torch.no_grad():
    output = gpt(torch.tensor([original_token_ids]))
print(output)

tensor([[[ 0.2267,  0.9132,  1.0494,  ..., -0.2330, -0.3008, -1.1458],
         [ 0.6808, -0.0495,  0.8574,  ...,  0.0671,  0.5572, -0.7873],
         [-0.1947,  0.1045,  0.2773,  ...,  1.3368,  0.8479, -0.9660],
         ...,
         [ 0.1200, -0.1027,  2.0549,  ..., -0.1519, -0.2096,  0.5651],
         [-1.1920, -0.1819, -0.0981,  ..., -0.1108,  0.8435, -0.3771],
         [-1.0612, -0.5674,  0.3229,  ...,  0.8383, -0.7121, -0.4850]]])


- 次に、更新されたトークンで試してみましょう：

In [17]:
with torch.no_grad():
    output = gpt(torch.tensor([new_token_ids]))
print(output)

tensor([[[ 0.2267,  0.9132,  1.0494,  ..., -0.2330, -0.3008, -1.1458],
         [ 0.6808, -0.0495,  0.8574,  ...,  0.0671,  0.5572, -0.7873],
         [-0.1947,  0.1045,  0.2773,  ...,  1.3368,  0.8479, -0.9660],
         ...,
         [-0.0656, -1.2451,  0.7957,  ..., -1.2124,  0.1044,  0.5088],
         [-1.1561, -0.7380, -0.0645,  ..., -0.4373,  1.1401, -0.3903],
         [-0.8961, -0.6437, -0.1667,  ...,  0.5663, -0.5862, -0.4020]]])


- ご覧のとおり、モデルは拡張されたトークンセットで動作します
- 実際には、新しいトークンを含むデータでモデル（特に新しい埋め込み層と出力層）をファインチューニング（または継続的に事前訓練）したいと思うでしょう

**重み共有についての注記**

- モデルが重み共有を使用している場合、つまり埋め込み層と出力層が同じ重みを共有している場合（Llama 3 [link]と同様）、出力層の更新ははるかに簡単です
- この場合、単に埋め込み層から重みをコピーできます：

In [18]:
gpt.out_head.weight = gpt.tok_emb.weight

In [19]:
with torch.no_grad():
    output = gpt(torch.tensor([new_token_ids]))